socket编程

In [ ]:
import socket

# tcp
sk_tcp = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
#upd
sk_udp = socket.socket(socket.AF_INET,socket.SOCK_DGRAM)


sk_tcp.close()

sk_udp.close()

关于线程的使用

In [7]:
import threading


def say_sth(sth,num):
    print(sth)
    print(num)
    
    
#创建线程并启动
t = threading.Thread(target=say_sth,args=([1,2,3],10))
t.start()

# #获取当前(进程)所有线程
# print(threading.enumerate())



class YourClass(threading.Thread):
    #继承Thread类必须实现run方法，才能通过start方法来启动
    def run(self):
        print("do your job")


        
# t = YourClass()
# t.start()


[1, 2, 3]
10


使用锁解决资源竞争问题

In [19]:
import threading
g_number = 0

def method1(num):
    global g_number
    mutex.acquire()
    for i in range(num):
        g_number += 1   
    print("g_number in method1 is ",g_number)
    mutex.release()
    
def method2(num):
    global g_number
    mutex.acquire()
    for i in range(num):
        g_number += 1
    print("g_number in method2 is ",g_number)
    mutex.release()
    
mutex = threading.Lock()
t1 = threading.Thread(target=method1,args=(1000000,))
t2 = threading.Thread(target=method2,args=(1000000,))
t1.start()
t2.start()

t1.join()
t2.join()
print("g_number in main thread is ",g_number)

g_number in method1 is  1000000
g_number in method2 is  2000000
g_number in main thread is  2000000


上锁的代码要尽可能少

In [21]:
import threading
g_number = 0

def method1(num):
    global g_number
    for i in range(num):
        # 上锁的代码要尽可能少
        mutex.acquire()
        g_number += 1   
        mutex.release()
    print("g_number in method1 is ",g_number)
    
    
def method2(num):
    global g_number
    
    for i in range(num):
        mutex.acquire()
        g_number += 1
        mutex.release()
    print("g_number in method2 is ",g_number)
    
    
mutex = threading.Lock()
t1 = threading.Thread(target=method1,args=(1000000,))
t2 = threading.Thread(target=method2,args=(1000000,))
t1.start()
t2.start()

t1.join()
t2.join()
print("g_number in main thread is ",g_number)

g_number in method1 is  1951472
g_number in method2 is  2000000
g_number in main thread is  2000000


队列的使用----多进程共享数据

In [2]:
import multiprocessing
q = multiprocessing.Queue()  # 设定队列大小，一般不指定
q.put(2) # 如果满了会阻塞
q.put_nowait(3) # 不阻塞，满了会报异常
q.get() # 如果空会阻塞
q.get_nowait()  # 空了不会阻塞，但会报错
q.empty() # 空判断
q.full()  # 满判断



In [4]:
import multiprocessing
def down_load_data(queue):
    # 模拟从网络下载数据
    data = [1,2,3,4,5]
    for _ in data:
        queue.put(_)
    print("data is ok")
    

def analysis_data(queue):
    # 模拟分析数据
    data = list()
    while not queue.empty():
        data.append(queue.get())
    print("data analysis ok")
    print(data)

def main():
    queue = multiprocessing.Queue()
    p1 =  multiprocessing.Process(target=down_load_data,args=(queue,))
    p2 =  multiprocessing.Process(target=analysis_data,args=(queue,))
    

    p1.start()
    p2.start()
    
if __name__ == "__main__":
    # jupyter 的原故，这里起动进程没有什么显示，打代码放到pychar就有结果了
    main()


进程池

In [1]:
import time
import os
import multiprocessing

def work(index):
    print("proccess：{}, index：{} is finished".format(os.getpid(), index))
    time.sleep(3)


if __name__ == "__main__":
    """
    在windows中, 创建进程由于没有folk创建进程, 所以是通过重载自身模块来创建的, 
    如果Pool放在if __name__ == "__main__"外部, 就会进行无限递归
    
    代码在jupyter中没有效果
    """
    pool = multiprocessing.Pool(3)  # 最多开辟3个进程
    for i in range(10):
        pool.apply_async(work, (i,))  # 向池子添加任务
#     pool.close()  # 关闭进程池，不再接受新的请求
#     pool.join()  # 等待所有子进程运行结束,必须放在close之后
    print("------end------")
    

------end------


多进程的queue、锁

import queque 普通的队列，在多进程中没用
multiprocessing.Queue()  可在多进程中使用，但不可以在进程池中使用
multiprocessing.Manager().Queue()  可以在进程池中使用

多进程的锁也是一样
multiprocessing.Manager().Lock()  可以在进程池中使用

迭代器

In [2]:
# 一个对象是否可迭代，主要看是否派生自Iterable
from collections import Iterable
print(isinstance("abcd",Iterable))

# 要实现一个可迭代的对象，必须实现__iter__方法，__iter__方法的返回值就是一个迭代器，要实现一个迭代器，又必须同时实现__iter__和__next__方
# 法

class Workmate(object):
    
    def __init__(self):
        self.names = list()
        self.index = 0
    def add(self,name):
        self.names.append(name)
        
    def __iter__(self): # 会被 iter()方法自动调用，遍历的时候也会被自动调用，返回一个可迭代的对像，Workmate自身就是可迭代的
        return self
    
    def __next__(self):   # 会被 next()方法自动调用，注意这里会存在下标越界问题，要判断是否小于len
        if self.index < len(self.names):
            ret = self.names[self.index]  
            self.index += 1
            return ret
        else: # 在循环中，__next__会一直被调用，就算下标越界了，它也不知道什么时候停止，所以一定要抛出StopIteration异常，for循环才会停
            raise StopIteration



workmate = Workmate()
workmate.add("张三")
workmate.add("李四")
workmate.add("王五")
    
for temp in workmate:
    print(temp)

True
张三
李四
王五


生成器，是一种特殊的迭代器
方式1 把列表推导式的中括号换成小括号，就得到了一个生成器   

In [9]:
ls = [x for x in range(10)]  # 这种形式很占空间
temp =(x for x in range(10)) # 这种形式不占空间
for num in temp:
    print(num)

0
1
2
3
4
5
6
7
8
9


方式2 只要方法里面有yield，就是一个生成器

如果这种方式生成的生成器里面有return一个结果，通常这个结果是拿不到的，通过next()一直迭代下去，到最后会报异常，try之后拿到异常对像ex
通过ex.value可以拿到return的结果

In [2]:
def fibonacii(number):
    """用生成器来生成斐波那契数列"""
    a, b = 0, 1
    counter = 0
    while counter < number:
        yield a
        a, b = b, a + b
        counter += 1
        
temp = fibonnacii(10)  # 这里就变成了创建生成器，而不是调用普通函数
for _ in temp:
    #print(_)
    pass

除了next()方法可以调用用yield生成的生成器，send()也可以调用生成器，并且可以向生成器传入参数。但，send()方法首次调用生成器传参只能传None，否则会报错。

In [8]:
def fibonacii(number):
    """用生成器来生成斐波那契数列"""
    a, b = 0, 1
    counter = 0
    while counter < number:
        ret  = yield a
        print("ret = ", ret)
        a, b = b, a + b
        counter += 1
        
        
obj = fibonacii(10)
# print(obj.send("your args"))  会报错，因为没有任何变量去接send()传入的参数
print(next(obj)) 
result = obj.send("hello world") 
# 在第一次调next()之后，程序在yield a处将值传出并暂停，send将参数传入给里面的ret，然后程序继续往前走，
# (你可以用拿到的参数修改原业务，不修改也可以)。在下一次yield a的时候，参值返回给外面，同时将a的值赋给ret，所以send的返回值是1
print(result)


0
ret =  hello world
1


In [ ]:
用yield实现多任务(假的多线程)

In [ ]:
import time
def task_1():
    while True:
        print("--------task_1-----------")
        time.sleep(1)
        yield # 重点是这里
        
def task_2():
    while True:
        print("--------task_2-----------")
        time.sleep(1)
        yield
        
def main():
    t1 = task_1()
    t2 = task_2()
    while True:
        next(t1)
        next(t2)
    

greenlet实现多任务--知道可以这么用而矣

In [3]:
from greenlet import greenlet
import time

def task1():
    while True:
        print("task1")
        g2.switch()
        time.sleep(1)
        
def task2():
    while True:
        print("task2")
        g1.switch()
        time.sleep(1)
        
g1 = greenlet(task1)
g2 = greenlet(task2)
# 切换到g1中运行
g1.switch()


gevent实现多任务（重点）,gevent是对greenlet的进一步封装！  
gevent一遇到耗时操作(gevent.sleep(1))，就会马上切换到下一个任务。  
如果不想用gevent.sleep()，要给代码先打包。

In [2]:
import gevent
import time
from gevent import monkey
import random

# 打包代码，将程序中所有耗时操作替换为gevent自己的实现，如time.sleep(1)会变成 gevent.sleep(1)
monkey.patch_all()

def task(n):
    for i in range(5):
        print(n, i)
        time.sleep(0.5)

# joinall 启动任务并等等所有任务运行完 如C#中的Task,WaitAll是一个意思
gevent.joinall([ 
    gevent.spawn(task,"task1"),# 新建任务并启动
    gevent.spawn(task,"task2"),
    gevent.spawn(task,"task3")
])
